In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import tqdm
from pandas.errors import SettingWithCopyWarning
import warnings
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
from include.beauti_print import *

<div class="center_header">

# Dataset building

</div>
<p>
I have collected the required data for the project from two website.

1. __[the sports db](https://www.thesportsdb.com/free_sports_api)__
1. __[number fire](https://www.numberfire.com)__

</p>

<h5 style="margin: 0px; padding: 0px;">the sports db:</h5>



you can download the dataset from __[here](dataset\basketball_players.csv)__

<style>
  .center_header {
  line-height: 90px;
  height: 90px;
  border: 5px solid blue;
  text-align: center;
  font-size: xx-large;
  }
</style>